## Litteraturbanken Author 
* this [notebook](https://github.com/salgo60/open-data-examples/blob/master/Litteraturbanken%20Author.ipynb)
* API [Author](https://litteraturbanken.se/api/get_authors)

In [58]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None) 
 
url = "https://litteraturbanken.se/api/get_authors"
r = http.request('GET', url)
data = json.loads(r.data)
df = pd.DataFrame(data["data"])

In [66]:
df.head(10)

,authorid,authorid_norm,birth,db_checksum,db_timestamp,death,doc_type,full_name,gender,imported,intro,librisid,name_for_index,pictureinfo,searchable,show,surname,updated,wikidata,db_timestamp_updated,dramawebben,popularity,pseudonym,sources,intro_author,intro_author_norm,picture,bibliography,external_ref,presentation,seemore,other_name
0,AarneA,AarneA,"{'date': '1867', 'plain': '1867'}",d4646f71d18e1304e4b16a557dd2f8cf,1582830514112,"{'date': '1925', 'plain': '1925'}",author,Antti Aarne,male,2014-02-11,None,0xbfm5hj2hjvvwn,"Aarne, Antti",None,False,False,Aarne,2019-10-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AasenE,AasenE,"{'date': '1935', 'plain': '1935'}",f9d8d6666830bbe11a5153518eb5ca7b,1582830510945,NaN,author,Elisabeth Aasen,female,2014-03-13,None,53hlsf1p1mc2mpw,"Aasen, Elisabeth",None,False,True,Aasen,2019-10-16,"{'birthplace': None, 'birthplace_label': None,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AbeniusM,AbeniusM,"{'date': '1899', 'plain': '1899'}",3f54cad107a4eabdbe746046f109a593,1582830510948,{'plain': '1970'},author,Margit Abenius,female,2017-04-14,\n<p>MARGIT ABENIUS (1899–1970): ”Margit Abeni...,ljx00mt45v0dfx5,"Abenius, Margit",None,True,True,Abenius,2019-10-16,"{'birthplace': [14.628893758, 61.090146823], '...",1.582831e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AbrahamssonG,AbrahamssonG,"{'date': '1961', 'plain': '1961'}",3d398cb6d33ac0cb9f1f0bc26d6eb9bf,1582830510957,NaN,author,Gunnel Abrahamsson,female,2014-02-11,None,NaN,"Abrahamsson, Gunnel",None,False,False,Abrahamsson,2014-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AbrahamssonS,AbrahamssonS,"{'date': '1872', 'plain': '1872'}",4f3cceeb16912e0524fd79011dafbf7f,1582830510959,"{'date': '1911', 'plain': '1911'}",author,Selma Abrahamsson,female,2018-10-25,None,NaN,"Abrahamsson, Selma",None,False,True,Abrahamsson,2018-10-25,NaN,1.582831e+12,{'intro': ' <p>SELMA ABRAHAMSSON (1872–1911) ...,384.0,"[{'authorid': 'Flavia_pseudonym', 'authorid_no...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AchariusE,AchariusE,"{'date': '1757', 'plain': '1757'}",77686e092dea51358a35a13a820195c8,1582830510962,"{'date': '1819', 'plain': '1819'}",author,Erik Acharius,male,2018-02-01,"\n<p>ERIK ACHARIUS (1757–1819): ”botaniker, Li...",dbqtx0hx5srsg85,"Acharius, Erik",None,False,True,Acharius,2019-10-16,"{'birthplace': [17.138055555, 60.672777777], '...",1.582831e+12,NaN,14.0,NaN,"[Bernhard Meijer, <a href=""/forfattare/MeijerB...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AdamBremen,AdamBremen,"{'date': '1066', 'plain': '0000'}",4ec10eece1b1cf51d645b0966da4c006,1582830510964,{'plain': '1095'},author,Adam av Bremen,male,2014-02-05,None,53hkk5vp1k798w1,Adam av Bremen,None,True,True,Adam av Bremen,2019-10-16,"{'birthplace': [13.4775, 51.163611111], 'birth...",1.582831e+12,NaN,12096.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Addäos,Addaos,"{'date': '-350', 'plain': '0000'}",da5e35f5f2e32602a17344e44b2b6e93,1582830510966,{'plain': '0000'},author,Addäos,male,2014-08-22,None,NaN,Addäos,None,False,True,Addäos,2016-04-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AdelborgO,AdelborgO,"{'date': '1855', 'plain': '1855'}",38af8810c6c130048aec525bfbbec82b,1582830510968,"{'date': '1936', 'plain': '1936'}",author,Ottilia Adelborg,female,2008-11-11,\n<p>\nOTTILIA ADELBORG (1855–1936) fick en ge...,sq466t3b4s743md,"Adelborg, Ottilia",None,True,True,Adelborg,2019-10-16,"{'birthplace': [15.588055555, 56.161111111], '...",1.582831e+12,{'intro': '<p>OTTILIA ADELBORG skrev flera bar...,15440.0,NaN,NaN,AlfredsonC,AlfredsonC,NaN,NaN,NaN,NaN,NaN,NaN
9,AdlerbethGJ,AdlerbethGJ,"{'date': '1751', 'plain': '1751'}",d3e4c674e2238333cbd8769a5714623d,1582830510970,"{'date': '1818', 'plain': '1818'}",author,Gudmund Jöran Adlerbeth,male,2010-10-19,\n\n\n<p>GUDMUND JÖRAN ADLERBETH (1751–1818) b...,wt794lsf0mx0xqp,"Adlerbeth, Gudmund Jöran","Porträttet är hämtat ur A. I. Arvidsson, ”Sven...",True,True,Adlerbeth,2019-10-16,"{'birthplace': [14.179167,

In [60]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2963 entries, 0 to 2962
Data columns (total 32 columns):
authorid                2963 non-null object
authorid_norm           2963 non-null object
birth                   2963 non-null object
db_checksum             2963 non-null object
db_timestamp            2963 non-null int64
death                   2262 non-null object
doc_type                2963 non-null object
full_name               2963 non-null object
gender                  2963 non-null object
imported                2291 non-null object
intro                   704 non-null object
librisid                1503 non-null object
name_for_index          2963 non-null object
pictureinfo             122 non-null object
searchable              2963 non-null bool
show                    2963 non-null bool
surname                 2963 non-null object
updated                 2291 non-null object
wikidata                1418 non-null object
db_timestamp_updated    1562 non-null float64

In [61]:
df.head(15)

,authorid,authorid_norm,birth,db_checksum,db_timestamp,death,doc_type,full_name,gender,imported,intro,librisid,name_for_index,pictureinfo,searchable,show,surname,updated,wikidata,db_timestamp_updated,dramawebben,popularity,pseudonym,sources,intro_author,intro_author_norm,picture,bibliography,external_ref,presentation,seemore,other_name
0,AarneA,AarneA,"{'date': '1867', 'plain': '1867'}",d4646f71d18e1304e4b16a557dd2f8cf,1582830514112,"{'date': '1925', 'plain': '1925'}",author,Antti Aarne,male,2014-02-11,None,0xbfm5hj2hjvvwn,"Aarne, Antti",None,False,False,Aarne,2019-10-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AasenE,AasenE,"{'date': '1935', 'plain': '1935'}",f9d8d6666830bbe11a5153518eb5ca7b,1582830510945,NaN,author,Elisabeth Aasen,female,2014-03-13,None,53hlsf1p1mc2mpw,"Aasen, Elisabeth",None,False,True,Aasen,2019-10-16,"{'birthplace': None, 'birthplace_label': None,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AbeniusM,AbeniusM,"{'date': '1899', 'plain': '1899'}",3f54cad107a4eabdbe746046f109a593,1582830510948,{'plain': '1970'},author,Margit Abenius,female,2017-04-14,\n<p>MARGIT ABENIUS (1899–1970): ”Margit Abeni...,ljx00mt45v0dfx5,"Abenius, Margit",None,True,True,Abenius,2019-10-16,"{'birthplace': [14.628893758, 61.090146823], '...",1.582831e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AbrahamssonG,AbrahamssonG,"{'date': '1961', 'plain': '1961'}",3d398cb6d33ac0cb9f1f0bc26d6eb9bf,1582830510957,NaN,author,Gunnel Abrahamsson,female,2014-02-11,None,NaN,"Abrahamsson, Gunnel",None,False,False,Abrahamsson,2014-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AbrahamssonS,AbrahamssonS,"{'date': '1872', 'plain': '1872'}",4f3cceeb16912e0524fd79011dafbf7f,1582830510959,"{'date': '1911', 'plain': '1911'}",author,Selma Abrahamsson,female,2018-10-25,None,NaN,"Abrahamsson, Selma",None,False,True,Abrahamsson,2018-10-25,NaN,1.582831e+12,{'intro': ' <p>SELMA ABRAHAMSSON (1872–1911) ...,384.0,"[{'authorid': 'Flavia_pseudonym', 'authorid_no...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AchariusE,AchariusE,"{'date': '1757', 'plain': '1757'}",77686e092dea51358a35a13a820195c8,1582830510962,"{'date': '1819', 'plain': '1819'}",author,Erik Acharius,male,2018-02-01,"\n<p>ERIK ACHARIUS (1757–1819): ”botaniker, Li...",dbqtx0hx5srsg85,"Acharius, Erik",None,False,True,Acharius,2019-10-16,"{'birthplace': [17.138055555, 60.672777777], '...",1.582831e+12,NaN,14.0,NaN,"[Bernhard Meijer, <a href=""/forfattare/MeijerB...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AdamBremen,AdamBremen,"{'date': '1066', 'plain': '0000'}",4ec10eece1b1cf51d645b0966da4c006,1582830510964,{'plain': '1095'},author,Adam av Bremen,male,2014-02-05,None,53hkk5vp1k798w1,Adam av Bremen,None,True,True,Adam av Bremen,2019-10-16,"{'birthplace': [13.4775, 51.163611111], 'birth...",1.582831e+12,NaN,12096.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Addäos,Addaos,"{'date': '-350', 'plain': '0000'}",da5e35f5f2e32602a17344e44b2b6e93,1582830510966,{'plain': '0000'},author,Addäos,male,2014-08-22,None,NaN,Addäos,None,False,True,Addäos,2016-04-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AdelborgO,AdelborgO,"{'date': '1855', 'plain': '1855'}",38af8810c6c130048aec525bfbbec82b,1582830510968,"{'date': '1936', 'plain': '1936'}",author,Ottilia Adelborg,female,2008-11-11,\n<p>\nOTTILIA ADELBORG (1855–1936) fick en ge...,sq466t3b4s743md,"Adelborg, Ottilia",None,True,True,Adelborg,2019-10-16,"{'birthplace': [15.588055555, 56.161111111], '...",1.582831e+12,{'intro': '<p>OTTILIA ADELBORG skrev flera bar...,15440.0,NaN,NaN,AlfredsonC,AlfredsonC,NaN,NaN,NaN,NaN,NaN,NaN
9,AdlerbethGJ,AdlerbethGJ,"{'date': '1751', 'plain': '1751'}",d3e4c674e2238333cbd8769a5714623d,1582830510970,"{'date': '1818', 'plain': '1818'}",author,Gudmund Jöran Adlerbeth,male,2010-10-19,\n\n\n<p>GUDMUND JÖRAN ADLERBETH (1751–1818) b...,wt794lsf0mx0xqp,"Adlerbeth, Gudmund Jöran","Porträttet är hämtat ur A. I. Arvidsson, ”Sven...",True,True,Adlerbeth,2019-10-16,"{'birthplace': [14.179167,

In [62]:
df.librisid.value_counts()

hftwzq01397wbbd    2
64jmq82q3rs896w    2
dbqsnnnx5flknct    2
xv8bc20g0w43850    1
20dgk82l4xwhzq6    1
                  ..
31fhk10m19x1w9g    1
wt79b08f1b6h1b8    1
1zcff24k2wtn4mm    1
hftx2c811gxpqdq    1
khwz0mh30l0r13m    1
Name: librisid, Length: 1500, dtype: int64

In [63]:
Wikidatadf = df["wikidata"]

## Wikidata